# Running MMS-ASR inference in Colab

In this notebook, we will give an example on how to run simple ASR inference using MMS ASR model. 

Credit to epk2112 [(github)](https://github.com/epk2112/fairseq_meta_mms_Google_Colab_implementation)

## Step 1: Clone fairseq-py and install latest version

In [7]:
!mkdir "temp_dir"
!git clone https://github.com/pytorch/fairseq

# Change current working directory
!pwd
%cd "/content/fairseq"
!pip install --editable ./ 
!pip install tensorboardX


fatal: destination path 'fairseq' already exists and is not an empty directory.
/content/fairseq
/content/fairseq
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/fairseq
  Installing build dependencies ... canceledERROR: Operation cancelled by user
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 2. Download MMS model
Un-comment to download your preferred model.
In this example, we use MMS-FL102 for demo purposes.
For better model quality and language coverage, user can use MMS-1B-ALL model instead (but it would require more RAM, so please use Colab-Pro instead of Colab-Free).


In [2]:
# MMS-1B:FL102 model - 102 Languages - FLEURS Dataset
!wget -P ./models_new 'https://dl.fbaipublicfiles.com/mms/asr/mms1b_fl102.pt'

# # MMS-1B:L1107 - 1107 Languages - MMS-lab Dataset
# !wget -P ./models_new 'https://dl.fbaipublicfiles.com/mms/asr/mms1b_l1107.pt'

# # MMS-1B-all - 1162 Languages - MMS-lab + FLEURS + CV + VP + MLS
# !wget -P ./models_new 'https://dl.fbaipublicfiles.com/mms/asr/mms1b_all.pt'

--2023-05-25 23:53:33--  https://dl.fbaipublicfiles.com/mms/asr/mms1b_fl102.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.227.219.33, 13.227.219.59, 13.227.219.70, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.227.219.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4851043301 (4.5G) [binary/octet-stream]
Saving to: ‘./models_new/mms1b_fl102.pt’

mms1b_fl102.pt      100%[===================>]   4.52G   242MB/s    in 20s     

2023-05-25 23:53:53 (230 MB/s) - ‘./models_new/mms1b_fl102.pt’ saved [4851043301/4851043301]



## 3. Prepare audio file
Create a folder on path '/content/audio_samples/' and upload your .wav audio files that you need to transcribe e.g. '/content/audio_samples/audio.wav' 

Note: You need to make sure that the audio data you are using has a sample rate of 16kHz You can easily do this with FFMPEG like the example below that converts .mp3 file to .wav and fixing the audio sample rate

Here, we use a FLEURS english MP3 audio for the example.

In [3]:
!wget -P ./audio_samples/ 'https://datasets-server.huggingface.co/assets/google/fleurs/--/en_us/train/0/audio/audio.mp3'
!ffmpeg -y -i ./audio_samples/audio.mp3 -ar 16000 ./audio_samples/audio.wav

--2023-05-25 23:53:53--  https://datasets-server.huggingface.co/assets/google/fleurs/--/en_us/train/0/audio/audio.mp3
Resolving datasets-server.huggingface.co (datasets-server.huggingface.co)... 50.17.173.235, 44.197.252.161, 3.216.183.114, ...
Connecting to datasets-server.huggingface.co (datasets-server.huggingface.co)|50.17.173.235|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20853 (20K) [audio/mpeg]
Saving to: ‘./audio_samples/audio.mp3’

audio.mp3           100%[===================>]  20.36K  --.-KB/s    in 0.09s   

2023-05-25 23:53:53 (238 KB/s) - ‘./audio_samples/audio.mp3’ saved [20853/20853]

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-fil

# 4: Run Inference and transcribe your audio(s)


In the below example, we will transcribe a sentence in English.

To transcribe other languages: 
1. Go to [MMS README ASR section](https://github.com/facebookresearch/fairseq/tree/main/examples/mms#asr)
2. Open Supported languages link
3. Find your target languages based on Language Name column
4. Copy the corresponding Iso Code
5. Replace `--lang "eng"` with new Iso Code

To improve the transcription quality, user can use language-model (LM) decoding by following this instruction [ASR LM decoding](https://github.com/facebookresearch/fairseq/tree/main/examples/mms#asr)

In [4]:
import os

os.environ["TMPDIR"] = '/content/temp_dir'
os.environ["PYTHONPATH"] = "."
os.environ["PREFIX"] = "INFER"
os.environ["HYDRA_FULL_ERROR"] = "1"
os.environ["USER"] = "micro"

!python examples/mms/asr/infer/mms_infer.py --model "/content/fairseq/models_new/mms1b_fl102.pt" --lang "eng" --audio "/content/fairseq/audio_samples/audio.wav"


>>> preparing tmp manifest dir ...
>>> loading model & running inference ...
2023-05-25 23:54:02.330426: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 23:54:04.144981: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Input: /content/fairseq/audio_samples/audio.wav
Output: a tornado is a spinning colum of very low-pressure air which sucks it surrounding air inward and upward


# 5: Beam search decoding using a Language Model and transcribe audio file(s)


Since MMS is a CTC model, we can further improve the accuracy by running beam search decoding using a language model. 

While we have not open sourced the language models used in MMS (yet!), we have provided the details of the data and commands to used to train the LMs in the Appendix section of our paper.


For this tutorial, we will use a alternate English language model based on Common Crawl data which has been made publicly available through the efforts of [Likhomanenko, Tatiana, et al. "Rethinking evaluation in asr: Are our models robust enough?."](https://arxiv.org/abs/2010.11745). The language model can be accessed from the GitHub repository [here](https://github.com/flashlight/wav2letter/tree/main/recipes/rasr). 

In [10]:
! mkdir -p /content/lmdecode 

!wget -P /content/lmdecode  https://dl.fbaipublicfiles.com/wav2letter/rasr/tutorial/lm_common_crawl_small_4gram_prun0-6-15_200kvocab.bin # smaller LM 
!wget -P /content/lmdecode  https://dl.fbaipublicfiles.com/wav2letter/rasr/tutorial/lexicon.txt 

--2023-05-26 00:16:03--  https://dl.fbaipublicfiles.com/wav2letter/rasr/tutorial/lm_common_crawl_small_4gram_prun0-6-15_200kvocab.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.227.219.33, 13.227.219.70, 13.227.219.10, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.227.219.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2627163608 (2.4G) [application/octet-stream]
Saving to: ‘/content/lmdecode/lm_common_crawl_small_4gram_prun0-6-15_200kvocab.bin’

lm_common_crawl_sma 100%[===================>]   2.45G  27.5MB/s    in 94s     

2023-05-26 00:17:37 (26.8 MB/s) - ‘/content/lmdecode/lm_common_crawl_small_4gram_prun0-6-15_200kvocab.bin’ saved [2627163608/2627163608]

--2023-05-26 00:17:37--  https://dl.fbaipublicfiles.com/wav2letter/rasr/tutorial/lexicon.txt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.227.219.33, 13.227.219.10, 13.227.219.70, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfil


Install decoder bindings from [flashlight](https://github.com/flashlight/flashlight)


In [37]:
# Taken from https://github.com/flashlight/flashlight/blob/main/scripts/colab/colab_install_deps.sh 
# Install dependencies from apt
! sudo apt-get install -y libfftw3-dev libsndfile1-dev libgoogle-glog-dev libopenmpi-dev libboost-all-dev
# Install Kenlm
! cd /tmp && git clone https://github.com/kpu/kenlm && cd kenlm && mkdir build && cd build && cmake .. -DCMAKE_BUILD_TYPE=Release && make install -j$(nproc)

# Install Intel MKL 2020
! cd /tmp && wget https://apt.repos.intel.com/intel-gpg-keys/GPG-PUB-KEY-INTEL-SW-PRODUCTS-2019.PUB && \
    apt-key add GPG-PUB-KEY-INTEL-SW-PRODUCTS-2019.PUB
! sh -c 'echo deb https://apt.repos.intel.com/mkl all main > /etc/apt/sources.list.d/intel-mkl.list' && \
    apt-get update && DEBIAN_FRONTEND=noninteractive apt-get install -y --no-install-recommends intel-mkl-64bit-2020.0-088
# Remove existing MKL libs to avoid double linkeage
! rm -rf /usr/local/lib/libmkl*


Reading package lists... Done
Building dependency tree       
Reading state information... Done
libboost-all-dev is already the newest version (1.71.0.0ubuntu2).
libopenmpi-dev is already the newest version (4.0.3-0ubuntu1).
libsndfile1-dev is already the newest version (1.0.28-7ubuntu0.1).
The following additional packages will be installed:
  libfftw3-bin libfftw3-long3 libfftw3-quad3 libfftw3-single3 libgflags-dev
  libgflags2.2 libgoogle-glog0v5
Suggested packages:
  libfftw3-doc
The following NEW packages will be installed:
  libfftw3-bin libfftw3-dev libfftw3-long3 libfftw3-quad3 libfftw3-single3
  libgflags-dev libgflags2.2 libgoogle-glog-dev libgoogle-glog0v5
0 upgraded, 9 newly installed, 0 to remove and 35 not upgraded.
Need to get 4,289 kB of archives.
After this operation, 24.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 libfftw3-long3 amd64 3.3.8-2ubuntu1 [313 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 

In [38]:
! rm -rf flashlight
! git clone --recursive https://github.com/flashlight/flashlight.git
%cd flashlight
! git checkout 035ead6efefb82b47c8c2e643603e87d38850076 
%cd bindings/python 
! python3 setup.py install

%cd /content/fairseq 

Cloning into 'flashlight'...
remote: Enumerating objects: 24032, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (123/123), done.
remote: Total 24032 (delta 41), reused 111 (delta 24), pack-reused 23882
Receiving objects: 100% (24032/24032), 15.30 MiB | 2.64 MiB/s, done.
Resolving deltas: 100% (17089/17089), done.
/content/fairseq/flashlight
Note: switching to '035ead6efefb82b47c8c2e643603e87d38850076'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 035ead6e AdvancedInde

Next, we download an audio file from [People's speech](https://huggingface.co/datasets/MLCommons/peoples_speech) data. We will the audio sample from their 'dirty' subset which will be more challenging for the ASR model. 

In [12]:
!wget -O ./audio_samples/tmp.wav 'https://datasets-server.huggingface.co/assets/MLCommons/peoples_speech/--/dirty/train/0/audio/audio.wav'
!ffmpeg -y -i ./audio_samples/tmp.wav -ar 16000 ./audio_samples/audio_noisy.wav



--2023-05-26 00:26:41--  https://datasets-server.huggingface.co/assets/MLCommons/peoples_speech/--/dirty/train/0/audio/audio.wav
Resolving datasets-server.huggingface.co (datasets-server.huggingface.co)... 34.200.186.24, 3.216.183.114, 44.197.252.161, ...
Connecting to datasets-server.huggingface.co (datasets-server.huggingface.co)|34.200.186.24|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 386924 (378K) [application/octet-stream]
Saving to: ‘./audio_samples/tmp.wav’

./audio_samples/tmp 100%[===================>] 377.86K  1.07MB/s    in 0.3s    

2023-05-26 00:26:42 (1.07 MB/s) - ‘./audio_samples/tmp.wav’ saved [386924/386924]

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enabl

Let's listen to the audio file 


In [16]:
import IPython
IPython.display.display(IPython.display.Audio("./audio_samples/audio_noisy.wav"))
print("Trancript: limiting emotions that we experience mainly in our childhood which stop us from living our life just open freedom i mean trust and")

Trancript: limiting emotions that we experience mainly in our childhood which stop us from living our life just open freedom i mean trust and


Run inference with both greedy decoding and LM decoding

In [39]:
import os

os.environ["TMPDIR"] = '/content/temp_dir'
os.environ["PYTHONPATH"] = "."
os.environ["PREFIX"] = "INFER"
os.environ["HYDRA_FULL_ERROR"] = "1"
os.environ["USER"] = "micro"

print("======= WITHOUT LM DECODING=======")

!python examples/mms/asr/infer/mms_infer.py --model "/content/fairseq/models_new/mms1b_fl102.pt" --lang "eng" --audio "/content/fairseq/audio_samples/audio.wav" "/content/fairseq/audio_samples/audio_noisy.wav"

print("\n\n\n======= WITH LM DECODING=======")

# Note that the lmweight, wordscore needs to tuned for each LM 
# Using the same values may not be optimal
decoding_cmds = """
decoding.type=kenlm 
decoding.beam=500 
decoding.beamsizetoken=50 
decoding.lmweight=2.69
decoding.wordscore=2.8
decoding.lmpath=/content/lmdecode/lm_common_crawl_small_4gram_prun0-6-15_200kvocab.bin
decoding.lexicon=/content/lmdecode/lexicon.txt
""".replace("\n", " ")
!python examples/mms/asr/infer/mms_infer.py --model "/content/fairseq/models_new/mms1b_fl102.pt" --lang "eng" --audio "/content/fairseq/audio_samples/audio.wav" "/content/fairseq/audio_samples/audio_noisy.wav" \
    --extra-infer-args '{decoding_cmds}'


======= WITHOUT LM DECODING=======
>>> preparing tmp manifest dir ...
>>> loading model & running inference ...
2023-05-26 01:01:58.415006: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-26 01:02:00.361210: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Input: /content/fairseq/audio_samples/audio.wav
Output: a tornado is a spinning colum of very low-pressure air which sucks it surrounding air inward and upward
Input: /content/fairseq/audio_samples/audio_noisy.wav
Output: limiting emotions that weexperienced mainly in our childhood which stop us from living our lives in just open freedom and interust and
======= WITH LM DECODING=======
>>> preparing tmp manifest dir ...
>>> loading model & 